In [2]:
# %%
import time
import numpy as np
import pickle
from numpy.linalg import det

import CMINE_lib as CMINE
# from Guassian_variables import Data_guassian

import pandas as pd
from scipy.stats import multivariate_normal
import itertools

np.random.seed(37)
from scipy import stats
from sklearn.neighbors import KernelDensity

import math

import torch 
import torch.nn as nn
import torch.nn.functional as F

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'


In [3]:
def log_sum_exp(value, dim=None, keepdim=False):
    """Numerically stable implementation of the operation
    value.exp().sum(dim, keepdim).log()
    """
    # TODO: torch.max(value, dim=None) threw an error at time of writing
    if dim is not None:
        m, _ = torch.max(value, dim=dim, keepdim=True)
        value0 = value - m
        if keepdim is False:
            m = m.squeeze(dim)
        return m + torch.log(torch.sum(torch.exp(value0),
                                       dim=dim, keepdim=keepdim))
    else:
        m = torch.max(value)
        sum_exp = torch.sum(torch.exp(value - m))
        if isinstance(sum_exp, Number):
            return m + math.log(sum_exp)
        else:
            return m + torch.log(sum_exp)

In [4]:
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True

In [5]:
from modules.CMI import DR_CMI, CDL_CMI

In [6]:
Dim = 5
batch_size = 64
#dataset = CMINE.create_dataset_DGP( Dim=5, N=batch_size)
dataset = CMINE.create_dataset_DGP_binary_A( Dim=5, N=batch_size)
s_t = torch.from_numpy(dataset[0]).float().cuda()
s_next = torch.from_numpy(dataset[1]).float().cuda()
a = torch.from_numpy(dataset[2]).float().cuda()

In [7]:
sample_dim = 2*Dim

hidden_size = 15
learning_rate = 0.005
training_steps = 10

cubic = False 


In [8]:
def train_dr(N = 64, training_steps = 10):
    torch.cuda.empty_cache()
    model_dr = DR_CMI(sample_dim + 1, sample_dim, hidden_size).cuda()
    optimizer_dr = torch.optim.Adam(model_dr.parameters(), learning_rate)
    dr_est_values = []
    for step in range(training_steps):
        #batch_x, batch_y = sample_correlated_gaussian(rho, dim=sample_dim, batch_size = batch_size, to_cuda = True, cubic = cubic)
        dataset = CMINE.create_dataset_DGP_binary_A(Dim=Dim, N=N)
        s_t = torch.from_numpy(dataset[0]).float().cuda()
        s_next = torch.from_numpy(dataset[1]).float().cuda()
        a = torch.from_numpy(dataset[2]).float().cuda()

        batch_x = torch.cat([s_t,a], dim=1)
        batch_y = s_next
        model_dr.eval()
        drs = model_dr(batch_x, batch_y)
        #mi_est_values.append(cmi)
        dr_est_values.append(drs)
        model_dr.train() 

        model_loss = model_dr.learning_loss(batch_x, batch_y)

        optimizer_dr.zero_grad()
        model_loss.backward(retain_graph=True)
        optimizer_dr.step()

        del batch_x, batch_y
        torch.cuda.empty_cache()
    return dr_est_values

In [9]:
def train_cdl(N = 64, training_steps = 10):
    torch.cuda.empty_cache()
    model_cdl = CDL_CMI(sample_dim + 1, sample_dim, hidden_size).cuda()
    optimizer_cdl = torch.optim.Adam(model_cdl.parameters(), learning_rate)
    cdl_est_values = []
    for step in range(training_steps):
        #batch_x, batch_y = sample_correlated_gaussian(rho, dim=sample_dim, batch_size = batch_size, to_cuda = True, cubic = cubic)
        dataset = CMINE.create_dataset_DGP_binary_A(Dim=Dim, N=N)
        s_t = torch.from_numpy(dataset[0]).float().cuda()
        s_next = torch.from_numpy(dataset[1]).float().cuda()
        a = torch.from_numpy(dataset[2]).float().cuda()

        batch_x = torch.cat([s_t,a], dim=1)
        batch_y = s_next
        model_cdl.eval()
        cdl_cmi = model_cdl(batch_x, batch_y)
        cdl_est_values.append(cdl_cmi)
        model_cdl.train() 

        model_loss = model_cdl.learning_loss(batch_x, batch_y)

        optimizer_cdl.zero_grad()
        model_loss.backward(retain_graph=True)
        optimizer_cdl.step()

        del batch_x, batch_y
        torch.cuda.empty_cache()
    return cdl_est_values

In [10]:
cdl_est_values = train_cdl()


In [11]:
dr_est_values = train_dr()

In [12]:
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[3.45191729e-01 8.35393500e-01 7.07713468e-01 2.07197335e-01
 2.98565660e-01 7.50967733e-01 3.33454261e+02 2.17969536e+01
 1.25966369e+04 1.05666578e+02]
[1.87818014e+02 1.61204949e+04 1.15004988e+01 1.80590769e+00
 3.32061178e+01 1.02272229e+01 2.31201053e+02 1.23502585e+00
 2.16986355e+01 6.09959363e+00]


In [19]:
N = 128
cdl_est_values = train_cdl(N)
dr_est_values = train_dr(N)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[7.30239090e+00 7.05152936e+00 8.23540059e+00 5.79760796e+00
 7.20315365e+00 2.12945217e+05 1.50455859e+02 2.78672594e+02
 2.27193331e+02 7.37535593e+03]
[6.10654989e+03 1.55910844e+05 3.77279128e+04 1.58046943e+11
 3.84559755e+07 2.88183968e+09 1.70990210e+06 6.64867495e+05
 7.54625585e+06 3.09346798e+11]


In [21]:
N = 32
cdl_est_values = train_cdl(N)
dr_est_values = train_dr(N)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[ 2.22894816  0.53415195  0.17980211  0.13757467  0.99465214 65.07050759
  4.91550556 16.42847439  3.3649228   1.53744228]
[5.39664560e+03 5.35883037e+03 5.52185233e+03 5.56562032e+03
 5.17552405e+03 1.64288803e+05 5.02380312e+04 5.52713212e+07
 2.11297229e+10 1.16834368e+08]


In [23]:
N = 16
cdl_est_values = train_cdl(N)
dr_est_values = train_dr(N)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[4.78701690e+00 9.01703398e+02 7.79684412e+00 5.26387394e+00
 1.29594241e+01 1.77291970e+06 8.20365912e+01 9.48841043e+01
 1.01327541e+03 1.45746310e+02]
[1.51966064e+05 2.71351952e-01 7.09830511e-01 6.07903015e-01
 5.05081787e+00 1.48763132e-01 7.62969087e-01 4.27091030e-01
 2.83229789e+01 2.05515325e+02]


# Training Step

In [13]:
N = 64
training_step = 100
cdl_est_values = train_cdl(N, training_step)
dr_est_values = train_dr(N, training_step)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[8.54707185e+01 8.21731112e+01 1.37683768e+02 1.57721224e+06
 4.46917246e+02 2.35233406e+04 2.65686910e+06 2.71871902e+04
 6.23914329e+03 1.55254032e+03]
[9.01370726e+05 1.03552596e+06 1.13081412e+09 6.07882058e+09
 1.04576471e+06 4.47428913e+06 4.34843873e+10 2.69823834e+06
 2.19596710e+09 1.74958618e+08]


In [ ]:
N = 32
training_step = 100
cdl_est_values = train_cdl(N, training_step)
dr_est_values = train_dr(N, training_step)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

In [31]:
N = 526
training_step = 100
cdl_est_values = train_cdl(N, training_step)
dr_est_values = train_dr(N, training_step)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[1.50006065e+01 2.18871596e+02 1.10105989e+07 5.11114454e+02
 1.06051950e+02 9.00031509e+01 1.04145438e+04 3.52512225e+03
 5.52125692e+03 2.41971664e+04]
[7.68950225e+06 7.74432824e+06 7.91731634e+06 7.70377130e+06
 7.70934032e+06 7.88427070e+08 4.68894041e+16 1.52884448e+10
 1.34091251e+11 2.40433729e+10]


In [30]:
N = 526
training_step = 10
cdl_est_values = train_cdl(N, training_step)
dr_est_values = train_dr(N, training_step)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[2.41623130e-01 5.83663910e-01 2.62432172e-01 1.86141613e-01
 2.88988191e+02 2.00060656e+01 5.57397880e+00 2.69901884e+01
 2.13138771e-01 1.53580540e+00]
[1.34774063e+06 7.49846064e+03 7.87508115e+03 7.57209025e+03
 7.50388969e+03 5.57463735e+05 1.85607135e+04 3.51530271e+09
 1.84042864e+05 2.73516473e+06]
